In [12]:
import os
from PIL import Image
import glob
import cv2
import albumentations as alb
import numpy as np
import json

c:\Users\pc\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Human Faces dataset contains many images with same image name. So, below is the code that we have used for renaming the images. So, that it does'nt create confusion in the further processing.

In [3]:
os.getcwd()
collection = "Path to Actual Human_faces(Kaggle_dataset)"
for i, filename in enumerate(os.listdir(collection)):
    os.rename("Path to Actual Human_faces(Kaggle_dataset)/" + filename, "Path to Actual Human_faces(Kaggle_dataset)" + 'p'+str(i) + ".jpg")

'f:\\Deep_learning'

# Resizing the different sizes images to same size (640,480).
The reason behind this choosing particular size is that when we captures frames using cv2, then it captures frame in this size.

In [5]:
image_list = []
resized_images = []

for filename in glob.glob('data\\temp\\*.jpg'): #temp stores all images(different sizes images)
    img = Image.open(filename)
    image_list.append(img)

for image in image_list:
    image = image.resize((640, 480))
    resized_images.append(image)

for (i, new) in enumerate(resized_images):
    try:
        new.save('{}{}{}'.format('data\\images\\', i+1, '.jpg')) #images folder will store all images resized to same size.
    except:
        print(i)
        pass

img = cv2.imread('data\\images\\1.jpg')
print(img.shape)

(480, 640, 3)


# Then , we manually labelled Images using Labelme

You can use !pip install labelme to resolve the dependecy

In [11]:
!labelme  #use this to label the images and save label(json file) in the Label folder.

# if this does not open labelme module then use cmd and type labelme to see the module.

# Data augmentation {using albumentations module}

In [12]:
augmentor = alb.Compose([
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.4),
                         alb.RandomGamma(p=0.2),                                          # These are the operation that we want to do on images for augmenting them.
                         alb.RGBShift(p=0.6), 
                         alb.VerticalFlip(p=0.5)], 
                         bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [11]:
for partition in ['train','test','val']: 
    print(os.listdir(os.path.join('data', partition, 'images')))
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.001,0.001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
    
        with open(label_path, 'r') as f:
            label = json.load(f)

        coords[0] = label['shapes'][0]['points'][0][0]
        coords[1] = label['shapes'][0]['points'][0][1]
        coords[2] = label['shapes'][0]['points'][1][0]
        coords[3] = label['shapes'][0]['points'][1][1]
        coords = list(np.divide(coords, [640,480,640,480]))  # albumentations accepts coordinates in the range(0-1).

        try: 
            for x in range(10):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['forehead'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)